In [ ]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, roc_curve, auc

In [ ]:
# Define transformations for training data with augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize to 224x224 pixels
    transforms.RandomHorizontalFlip(p=0.5),       # Randomly flip images horizontally
    transforms.RandomRotation(degrees=15),        # Randomly rotate within ±15 degrees
    transforms.ColorJitter(brightness=0.2,        # Adjust brightness, contrast, saturation, and hue
                           contrast=0.2,
                           saturation=0.2,
                           hue=0.1),
    transforms.ToTensor(),                        # Convert image to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Define transformations for validation and test data (no augmentation)
test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize to 224x224 pixels
    transforms.ToTensor(),                        # Convert image to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

In [ ]:
# Load datasets with respective transformations
train_dataset = datasets.ImageFolder(
    root='/kaggle/input/tumortrace/clasification-roi/train',  # Update the path to your dataset
    transform=train_transform
)
val_dataset = datasets.ImageFolder(
    root='/kaggle/input/tumortrace/clasification-roi/val',  # Update the path to your dataset
    transform=test_val_transform
)
test_dataset = datasets.ImageFolder(
    root='/kaggle/input/tumortrace/clasification-roi/test',  # Update the path to your dataset
    transform=test_val_transform
)

In [ ]:
# Create DataLoaders for each dataset
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Display class names in the dataset
print(f"Available class names: {train_dataset.classes}")

In [ ]:
# Function to count images in each class
def count_classes(dataset):
    class_counts = {class_name: 0 for class_name in dataset.classes}

    for _, labels in DataLoader(dataset, batch_size=1):
        for label in labels:
            class_name = dataset.classes[label.item()]
            class_counts[class_name] += 1

    return class_counts

# Count images in each dataset
train_counts = count_classes(train_dataset)
val_counts = count_classes(val_dataset)
test_counts = count_classes(test_dataset)

# Print the counts
print(f"Train dataset counts: {train_counts}")
print(f"Validation dataset counts: {val_counts}")
print(f"Test dataset counts: {test_counts}")

In [ ]:
# Function to convert tensor back to PIL image
def tensor_to_pil(tensor):
    unnormalize = transforms.Normalize(
        mean=[-0.5 / 0.5, -0.5 / 0.5, -0.5 / 0.5],
        std=[1 / 0.5, 1 / 0.5, 1 / 0.5]
    )
    tensor = unnormalize(tensor).clamp(0, 1)  # Undo normalization and clamp to valid range
    return transforms.ToPILImage()(tensor)    # Convert tensor to PIL image

In [ ]:
# Function to display augmented images
def show_augmented_images(dataset, class_name, num_images=5):
    if class_name not in dataset.classes:
        raise ValueError(f"Class '{class_name}' not found. Available classes: {dataset.classes}")

    plt.figure(figsize=(15, 5))  # Create figure

    # Find the class index for the requested class
    class_idx = dataset.class_to_idx[class_name]

    # Load a sample image of the requested class
    for img, label in DataLoader(dataset, batch_size=1, shuffle=True):
        if label.item() == class_idx:  # Check if the image matches the desired class
            sample_img = img[0]  # Extract the image tensor
            pil_img = tensor_to_pil(sample_img)  # Convert tensor to PIL image
            break

    # Display multiple augmented versions of the same image
    for i in range(num_images):
        augmented_img = train_transform(pil_img)  # Apply random transformations again

        plt.subplot(1, num_images, i + 1)  # Create a subplot for each image
        plt.imshow(augmented_img.permute(1, 2, 0).numpy())  # Convert tensor to numpy
        plt.axis('off')  # Hide axes

    plt.suptitle(f'Augmented versions of class: {class_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# Display augmented images for the first class
show_augmented_images(train_dataset, class_name=train_dataset.classes[0], num_images=5)

# If there are multiple classes, display augmented images for the second class
if len(train_dataset.classes) > 1:
    show_augmented_images(train_dataset, class_name=train_dataset.classes[1], num_images=5)

In [ ]:
# Import libraries for image processing and visualization
import cv2
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Load and resize an image
def load_and_resize_image(image_path, size=(224, 224)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    return cv2.resize(image, size)  # Resize to target dimensions

In [ ]:
# Plot resized image and pixel intensity histogram
def plot_image_and_histogram(image):
    pixel_values = image.flatten()
    
    plt.figure(figsize=(12, 6))
    
    # Plot the resized image
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Resized Image')
    plt.axis('off')
    
    # Plot the histogram
    plt.subplot(1, 2, 2)
    plt.hist(pixel_values, bins=256, range=(0, 256), color='gray', alpha=0.7)
    plt.title('Histogram of Pixel Values')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Compute and plot HOG features
def plot_hog_features(image):
    hog_features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    
    plt.figure(figsize=(12, 6))
    
    # Original image
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    # HOG visualization
    plt.subplot(1, 2, 2)
    plt.imshow(hog_image_rescaled, cmap='gray')
    plt.title('HOG Visualization')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
image_path = "/kaggle/input/tumortrace/clasification-roi/test/Benign/BreaDM-Be-1810/SUB1/p-030.jpg"  # Update with a valid path
image_resized = load_and_resize_image(image_path)

# Visualize
plot_image_and_histogram(image_resized)
plot_hog_features(image_resized)

In [ ]:
def convolve(image, kernel):
    """Perform convolution on an image with a kernel."""
    image_x, image_y = image.shape
    kernel_x, kernel_y = kernel.shape

    height_radius, width_radius = np.array(kernel.shape) // 2
    output = np.zeros(image.shape)
    
    # Stride and padding
    stride = 1
    padding = 0
    output_x = int(((image_x - kernel_x + 2 * padding) // stride) + 1)
    output_y = int(((image_y - kernel_y + 2 * padding) // stride) + 1)
    
    print(f"Output Dimensions: {output_x}x{output_y}")

In [ ]:
def compute_lbp(image):
    """Compute the Local Binary Pattern of an image."""
    height, width = image.shape
    lbp_image = np.zeros((height - 2, width - 2), dtype=np.uint8)  # Initialize LBP image

    # Iterate over each pixel (excluding the border)
    for i in range(1, height - 1):
        for j in range(1, width - 1):
            center_pixel = image[i, j]
            binary_string = ''  # Initialize binary string for LBP pattern
            
            # Check the 8 neighbors in clockwise order
            binary_string += '1' if image[i-1, j-1] >= center_pixel else '0'  # Top-left
            binary_string += '1' if image[i-1, j] >= center_pixel else '0'    # Top-center
            binary_string += '1' if image[i-1, j+1] >= center_pixel else '0'  # Top-right
            binary_string += '1' if image[i, j+1] >= center_pixel else '0'    # Middle-right
            binary_string += '1' if image[i+1, j+1] >= center_pixel else '0'  # Bottom-right
            binary_string += '1' if image[i+1, j] >= center_pixel else '0'    # Bottom-center
            binary_string += '1' if image[i+1, j-1] >= center_pixel else '0'  # Bottom-left
            binary_string += '1' if image[i, j-1] >= center_pixel else '0'    # Middle-left

            # Convert binary string to decimal and assign to the LBP image
            lbp_value = int(binary_string, 2)
            lbp_image[i-1, j-1] = lbp_value

    return lbp_image  # Return the computed LBP image

In [ ]:
def display_lbp(image_path):
    """Load an image, compute LBP, and display both."""
    image = load_and_resize_image(image_path)        # Load the image
    lbp_image = compute_lbp(image)         # Compute LBP

    # Plot the original and LBP images
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title('Original Image (224x224)')
    plt.imshow(image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title('Local Binary Pattern (LBP)')
    plt.imshow(lbp_image, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

display_lbp(image_path)

In [ ]:
def compute_mvm_lbp(image):
    """Compute Mean-Variance-Median Local Binary Pattern."""
    height, width = image.shape
    lbp_mean = np.zeros((height - 2, width - 2), dtype=np.uint8)
    lbp_variance = np.zeros((height - 2, width - 2), dtype=np.uint8)
    lbp_median = np.zeros((height - 2, width - 2), dtype=np.uint8)

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            window = image[i - 1:i + 2, j - 1:j + 2].flatten()
            neighbors = np.delete(window, 4)

            mean, variance, median = np.mean(neighbors), np.var(neighbors), np.median(neighbors)
            lbp_mean[i - 1, j - 1] = int(''.join(['1' if p >= mean else '0' for p in neighbors]), 2)
            lbp_variance[i - 1, j - 1] = int(''.join(['1' if p >= variance else '0' for p in neighbors]), 2)
            lbp_median[i - 1, j - 1] = int(''.join(['1' if p >= median else '0' for p in neighbors]), 2)

    return lbp_mean, lbp_variance, lbp_median

In [ ]:
# Load and preprocess the image
image_path = '/kaggle/input/tumortrace/clasification-roi/train/Malignant/BreaDM-Ma-2127/SUB7/p-049.jpg'

image = load_and_resize_image(image_path)

# Compute and Display LBP
lbp_image = compute_lbp(image)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image (224x224)')
plt.imshow(image, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(lbp_image, cmap='gray')
plt.title('Local Binary Pattern')
plt.axis('off')
plt.show()

# Compute and Display MVM-LBP
lbp_mean, lbp_variance, lbp_median = compute_mvm_lbp(image)

plt.figure(figsize=(15, 5))
plt.subplot(1, 4, 1)
plt.title('Original Image (224x224)')
plt.imshow(image, cmap='gray')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.title('Mean-LBP')
plt.imshow(lbp_mean, cmap='gray')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.title('Variance-LBP')
plt.imshow(lbp_variance, cmap='gray')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.title('Median-LBP')
plt.imshow(lbp_median, cmap='gray')
plt.axis('off')

plt.show()

In [ ]:
import numpy as np
from skimage import io, color, transform
import matplotlib.pyplot as plt

def resize_and_normalize(image, size=(224, 224)):
    """
    Resizes the image to the specified size and normalizes pixel values to 0-255.
    """
    # Resize the image
    image_resized = transform.resize(image, size, anti_aliasing=True)
    
    # Normalize to range [0, 255] and convert to uint8
    image_normalized = (image_resized * 255).astype(np.uint8)
    
    return image_normalized

def compute_glcm(image, distance=1, angle=0, levels=256):
    """
    Computes a Gray-Level Co-occurrence Matrix (GLCM) for a given grayscale image.
    
    Parameters:
    - image: The input grayscale image as a 2D NumPy array.
    - distance: Pixel distance for GLCM calculation (1 for adjacent pixels).
    - angle: Angle in degrees (0 for horizontal, 90 for vertical, 45 for diagonal).
    - levels: Number of gray levels (typically 256 for 8-bit images).
    
    Returns:
    - glcm: A 2D NumPy array representing the GLCM.
    """
    glcm = np.zeros((levels, levels), dtype=np.int32)
    rows, cols = image.shape
    
    # Define direction based on angle
    if angle == 0:  # Horizontal
        row_offset, col_offset = 0, distance
    elif angle == 90:  # Vertical
        row_offset, col_offset = distance, 0
    elif angle == 45:  # Diagonal (bottom-left to top-right)
        row_offset, col_offset = -distance, distance
    elif angle == 135:  # Diagonal (top-left to bottom-right)
        row_offset, col_offset = distance, distance
    else:
        raise ValueError("Angle must be 0, 45, 90, or 135 degrees.")
    
    # Calculate GLCM
    for i in range(rows):
        for j in range(cols):
            # Determine neighboring pixel based on offsets
            row_neighbor = i + row_offset
            col_neighbor = j + col_offset
            
            # Check if the neighboring pixel is within image boundaries
            if 0 <= row_neighbor < rows and 0 <= col_neighbor < cols:
                current_pixel = image[i, j]
                neighbor_pixel = image[row_neighbor, col_neighbor]
                
                # Accumulate GLCM counts
                glcm[current_pixel, neighbor_pixel] += 1
                
    return glcm

# Load the image
image = io.imread('/kaggle/input/tumortrace/clasification-roi/train/Malignant/BreaDM-Ma-2127/SUB7/p-049.jpg')  # Replace 'path_to_image' with the actual path

# Convert to grayscale if it's an RGB image
if image.ndim == 3:
    image = color.rgb2gray(image)

# Resize and normalize the image
image_processed = resize_and_normalize(image)

# Compute the horizontal GLCM with 1 pixel offset and 0 degrees (horizontal)
glcm = compute_glcm(image_processed, distance=1, angle=0)

# Print the GLCM matrix to inspect its raw values
print("Horizontal GLCM (0 degrees):")
print(glcm)

VGG MODEL

In [ ]:
# Import necessary PyTorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import roc_curve, auc as compute_auc, confusion_matrix , roc_auc_score
import numpy as np
import os

In [ ]:
# Load the pre-trained VGG-16 model
vgg16 = models.vgg16(pretrained=True)

# Set the model to evaluation mode
vgg16.eval()

# Print the VGG-16 architecture
print(vgg16)

In [ ]:
# Define a custom VGG-16 model for a specific number of output classes
class CustomVGG16(nn.Module):
    def __init__(self, num_classes=2):
        super(CustomVGG16, self).__init__()

        # Load the pre-trained VGG-16 model
        vgg16 = models.vgg16(pretrained=True)
        
        # Extract the features and avgpool layers from the pre-trained model
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool
        
        # Define a new classifier with custom layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # First fully connected layer
            nn.ReLU(inplace=True),         # ReLU activation
            nn.Dropout(p=0.5),             # Dropout layer
            nn.Linear(4096, 4096),         # Second fully connected layer
            nn.ReLU(inplace=True),         # ReLU activation
            nn.Dropout(p=0.5),             # Dropout layer
            nn.Linear(4096, num_classes)   # Output layer for classification
        )

    def forward(self, x):
        # Forward pass through the feature extraction layers
        x = self.features(x)
        
        # Forward pass through the average pooling layer
        x = self.avgpool(x)
        
        # Flatten the output from avgpool to match the input size of the classifier
        x = torch.flatten(x, 1)
        
        # Forward pass through the custom classifier
        x = self.classifier(x)
        
        return x

# ResNet18 Model
class ResNet18(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet18, self).__init__()
        model_resnet18 = models.resnet18(pretrained=True)
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool
        self.features = model_resnet18.fc.in_features
        self.fc = nn.Linear(self.features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# ResNet50 Model
class ResNet50(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet50, self).__init__()
        model_resnet50 = models.resnet50(pretrained=True)
        self.conv1 = model_resnet50.conv1
        self.bn1 = model_resnet50.bn1
        self.relu = model_resnet50.relu
        self.maxpool = model_resnet50.maxpool
        self.layer1 = model_resnet50.layer1
        self.layer2 = model_resnet50.layer2
        self.layer3 = model_resnet50.layer3
        self.layer4 = model_resnet50.layer4
        self.avgpool = model_resnet50.avgpool
        self.features = model_resnet50.fc.in_features
        self.fc = nn.Linear(self.features, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
# Create an instance of the custom VGG-16 model
model = CustomVGG16(num_classes=2)

# Print the architecture of the custom model
print(model)

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        score = -val_loss  # Minimizing val_loss, so higher "score" is better

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def reset(self):
        
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decreases."""
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# Model and Hyperparameter Initialization
num_classes = 2
lr = 0.01
momentum = 0.9
l2_decay = 0.01
total_epochs = 50
log_interval = 10

In [ ]:
# Training Function
def train(epoch, model, num_epochs, loader, criterion, l2_decay):
    learning_rate = max(lr * (0.1 ** (epoch // 10)), 1e-5)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=l2_decay)
    model.train()

    correct = 0
    for data, label in tqdm(loader, desc=f'Epoch {epoch}/{num_epochs}', unit='batch'):
        data, label = data.float().to(device), label.long().to(device)

        output = model(data)
        optimizer.zero_grad()
        loss = F.nll_loss(F.log_softmax(output, dim=1), label)
        loss.backward()
        optimizer.step()

        pred = output.data.max(1)[1]
        correct += pred.eq(label.data.view_as(pred)).cpu().sum()

    accuracy = 100. * correct / len(loader.dataset)
    print(f"Epoch {epoch}: Train Accuracy: {accuracy:.2f}%")

In [ ]:
# Validation Function
def validation(model, val_loader):
    model.eval()
    test_loss = 0
    correct = 0
    all_predictions, all_targets = [], []
    possibilities = None

    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        val_output = model(data)

        test_loss += F.nll_loss(F.log_softmax(val_output, dim=1), target, reduction='sum').item()
        pred = val_output.data.max(1)[1]
        all_predictions.extend(pred.cpu().numpy())
        all_targets.extend(target.cpu().numpy())

        possibility = F.softmax(val_output, dim=1).cpu().detach().numpy()
        possibilities = np.concatenate((possibilities, possibility), axis=0) if possibilities is not None else possibility

        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    cm = confusion_matrix(all_targets, all_predictions)

    num_classes = val_output.shape[1]
    label_onehot = np.eye(num_classes)[np.array(all_targets).astype(int)]
    fpr, tpr, _ = roc_curve(label_onehot.ravel(), possibilities.ravel())
    auc_score = compute_auc(fpr, tpr)

    test_loss /= len(val_loader.dataset)
    accuracy = 100. * correct / len(val_loader.dataset)
    specificity = 1 - fpr[1] if len(fpr) > 1 else 0
    sensitivity = tpr[1] if len(tpr) > 1 else 0

    print(f"Validation: Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%, AUC: {auc_score:.4f}")
    print(f"Confusion Matrix:\n{cm}")  # Print confusion matrix
    print(f"Specificity: {specificity:.4f}, Sensitivity: {sensitivity:.4f}")
    return test_loss, accuracy, cm, auc_score

In [ ]:
# Custom model (replace `customVGG16` with your model class)
model = CustomVGG16(num_classes=num_classes)
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Early stopping setup (implement EarlyStopping in your environment)
early_stop = EarlyStopping(patience=20, verbose=True)

# Project and model names
project_name = 'tumor_classification'
model_name = 'vgg16'

# Model Training and Evaluation
best_accuracy = 0
model_save_dir = os.path.join('model', project_name, model_name)
os.makedirs(model_save_dir, exist_ok=True)

for epoch in range(1, total_epochs + 1):
    train(epoch, model, total_epochs, train_loader, criterion, l2_decay)

    with torch.no_grad():
        test_loss, accuracy, cm, auc = validation(model, val_loader)

    # Save the model if it achieves the best AUC
    model_dict = model.module.state_dict() if isinstance(model, nn.parallel.DistributedDataParallel) else model.state_dict()
    if auc > best_accuracy:
        best_accuracy = auc
        torch.save(model_dict, os.path.join(model_save_dir, f'{model_name}_{epoch}.pth'))

    early_stop(test_loss, model)
    if early_stop.early_stop:
        print("Early stopping")
        break

In [ ]:
# Initialize ResNet18 Model
model = ResNet18(num_classes=num_classes)
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Early stopping setup (implement EarlyStopping in your environment)
early_stop = EarlyStopping(patience=20, verbose=True)

# Project and model names
project_name = 'tumor_classification'
model_name = 'Resenet18'

# Loss function
criterion = nn.CrossEntropyLoss()

best_accuracy = 0
model_save_dir = os.path.join('model', project_name, model_name)
os.makedirs(model_save_dir, exist_ok=True)


# Training loop
for epoch in range(1, total_epochs + 1):
    train(epoch, model, total_epochs, train_loader, criterion, l2_decay)
    with torch.no_grad():
        test_loss, accuracy, cm, auc = validation(model, val_loader)
    # Save the model if it achieves the best AUC
    model_dict = model.module.state_dict() if isinstance(model, nn.parallel.DistributedDataParallel) else model.state_dict()
    if auc > best_accuracy:
        best_accuracy = auc
        torch.save(model_dict, os.path.join(model_save_dir, f'{model_name}_{epoch}.pth'))
    early_stop(test_loss, model)
    if early_stop.early_stop:
        print("Early stopping")
        break

In [ ]:
# Initialize ResNet18 Model
model = ResNet50(num_classes=num_classes)
model = model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Early stopping setup (implement EarlyStopping in your environment)
early_stop = EarlyStopping(patience=15, verbose=True)

# Project and model names
project_name = 'tumor_classification'
model_name = 'Resenet50'

# Loss function
criterion = nn.CrossEntropyLoss()

best_accuracy = 0
model_save_dir = os.path.join('model', project_name, model_name)
os.makedirs(model_save_dir, exist_ok=True)


# Training loop
for epoch in range(1, total_epochs + 1):
    train(epoch, model, total_epochs, train_loader, criterion, l2_decay)
    with torch.no_grad():
        test_loss, accuracy, cm, auc = validation(model, val_loader)
    # Save the model if it achieves the best AUC
    model_dict = model.module.state_dict() if isinstance(model, nn.parallel.DistributedDataParallel) else model.state_dict()
    if auc > best_accuracy:
        best_accuracy = auc
        torch.save(model_dict, os.path.join(model_save_dir, f'{model_name}_{epoch}.pth'))
    early_stop(test_loss, model)
    if early_stop.early_stop:
        print("Early stopping")
        break

In [ ]:
import torch
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve
import torch.nn.functional as F
import matplotlib.pyplot as plt

def test_with_visualization(model, test_dataloader, device='cpu'):
    model.eval()

    test_loss = 0
    correct = 0
    possibilities = None
    all_predictions = []
    all_true_labels = []  # Store all true labels

    for data, target in test_dataloader:
        if torch.cuda.is_available():
            data, target = data.to(device), target.to(device)

        test_output = model(data)
        test_loss += F.nll_loss(F.log_softmax(test_output, dim=1), target, reduction='sum').item()

        pred = test_output.data.max(1)[1]
        all_predictions.append(pred.cpu().numpy())
        all_true_labels.append(target.cpu().numpy())

        possibility = F.softmax(test_output, dim=1).cpu().data.numpy()
        if possibilities is None:
            possibilities = possibility
        else:
            possibilities = np.concatenate((possibilities, possibility), axis=0)

        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    # Flatten all predictions and true labels
    all_predictions = np.concatenate(all_predictions)
    all_true_labels = np.concatenate(all_true_labels)

    # Classification metrics -> accuracy, f1 score
    print(f"Results for {model.__class__.__name__}:")
    print(metrics.classification_report(all_true_labels, all_predictions, target_names=['benign', 'malignant'], digits=4))

    # Confusion matrix
    cm = metrics.confusion_matrix(all_true_labels, all_predictions)
    print(f"Confusion Matrix for {model.__class__.__name__}:\n", cm)

    # Plot Confusion Matrix
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(len(set(all_true_labels)))
    plt.xticks(tick_marks, tick_marks)
    plt.yticks(tick_marks, tick_marks)
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

    # ROC curve and AUC
    num_classes = test_output.shape[1]
    label_onehot = np.eye(num_classes)[all_true_labels]

    fpr, tpr, _ = roc_curve(label_onehot.ravel(), possibilities.ravel())
    auc_value = roc_auc_score(label_onehot, possibilities, average="macro")

    # Plot ROC Curve
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_value:.4f})")
    plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower right")
    plt.show()

    test_loss /= len(test_dataloader.dataset)
    print(f'Specificity: {1 - fpr[0]:.4f}, Sensitivity: {tpr[0]:.4f}, AUC: {auc_value:.4f}')
    print(f'\n{model.__class__.__name__} set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_dataloader.dataset)} ({100. * correct / len(test_dataloader.dataset):.2f}%)\n')

    return 100. * correct / len(test_dataloader.dataset), test_loss, auc_value


In [ ]:
test_accuracy, test_loss, test_auc = test_with_visualization(model, test_loader, device='cuda' if torch.cuda.is_available() else 'cpu')

if test_accuracy is not None:
    print(f"Test Accuracy: {test_accuracy:.2f}%")
